In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Read the two images
img1 = cv2.imread(r'C:\Users\ML08\Desktop\27-04-2023\OUTPUT FOLDER\P01X010-2022111700190_low_final.png', -1)
img2 = cv2.imread(r'C:\Users\ML08\Desktop\27-04-2023\OUTPUT FOLDER\P01X010-2022111700190_high_final.png', -1)

# Calculate the absolute difference between the two grayscale images
diff_bg = cv2.absdiff(img1, img2)
diff_new = img1-img2
#cv2.imwrite('diff_new.png', diff_bg)
avg = (img1/2)+(img2/2)


# Create a mask of pixel values <= 10 in the difference image
threshold = 20
mask = np.zeros_like(diff_bg, dtype=np.uint16)
for i in range(diff_bg.shape[0]):
    for j in range(diff_bg.shape[1]):
        if diff_bg[i,j] < threshold:
            mask[i,j] = 1

# Replace pixel values in diff image <= 10 with values from avg image in corresponding positions
for i in range(diff_bg.shape[0]):
    for j in range(diff_bg.shape[1]):
        if mask[i,j] == 1:
            diff_bg[i,j] = avg[i,j]
        else:
            diff_bg[i,j] = 0
            
# Save the result
cv2.imwrite('background.png', diff_bg)


# Apply a 3x3 averaging filter to X
kernel = np.ones((3, 3), np.float32) / 9
R = cv2.filter2D(avg, -1, kernel)
#R = avg

# Create the detail image
detail = np.zeros_like(img1)

for i in range(img1.shape[0]):
    for j in range(img1.shape[1]):
        # Compute the absolute difference between Image1 and image2
        diff = np.abs(img1[i, j].astype(np.uint16) - img2[i, j].astype(np.uint16))
    
        # Check if the difference is greater than 10
        if diff > threshold:
            # Compute the absolute difference between R and each image
            diff1 = np.abs(R[i, j].astype(np.uint16) - img2[i, j].astype(np.uint16))
            diff2 = np.abs(R[i, j].astype(np.uint16) - img1[i, j].astype(np.uint16))
            
            # Check which image has the smaller absolute difference to R
            if diff1 < diff2:
                detail[i, j] = img1[i, j]
            else:
                detail[i, j] = img2[i, j]
        else:
            detail[i,j] = 0

final = detail+diff_bg
cv2.imwrite('detail.png', detail)
cv2.imwrite('final_after.png', final)


# Apply gamma correction
gamma = 2.2
final = cv2.convertScaleAbs(final, alpha=(255.0/65535.0))
gamma_img = np.power(final / 255.0, gamma) * 255.0
gamma_img = gamma_img.astype(np.uint8)

# Save the gamma-corrected image
cv2.imwrite("average_gamma.png", gamma_img)

img = np.zeros(img1.shape, dtype=np.uint16)

# Replace all pixels in the image with the value of 0.8
img[:] = 0.8*255
img = img.astype(np.uint8)

# Stack the images
stacked_img = np.stack((final, img, gamma_img), axis=-1).astype(np.uint8)

bgr_img = cv2.cvtColor(stacked_img, cv2.COLOR_HSV2BGR)

# Convert BGR to RGB
rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)

plt.imsave('before.png', rgb_img)

# Apply "hot" colormap
colormap_img = cv2.applyColorMap(stacked_img, cv2.COLORMAP_HOT)

# Convert the image to RGB
colormap_img_rgb = cv2.cvtColor(colormap_img.astype(np.uint8), cv2.COLOR_BGR2RGB)

# Save the image using Matplotlib
#cv2.imwrite('stacked_hot.png', colormap_img)
plt.imsave('stacked_hot_1.png', colormap_img_rgb)